# Prevendo o Nível de Satisfação dos Clientes do Santander

## Definição do Problema de Negócio

Identificar clientes insatisfeitos no início do relacionamento com o Banco Santander

## Importando pacotes necessários para o código

In [1]:
import pandas as pd #importando o pacote pandas
from sklearn.pipeline import Pipeline #importando Pipeline, para realizar as atividades em ordem
from sklearn.ensemble import ExtraTreesClassifier #importando as Random Forest para Feature Selection
from sklearn.feature_selection import RFE #Eliminação Recursiva
from sklearn.model_selection import GridSearchCV #importando GridSearchCV, para realizar tuning do modelo
from sklearn.preprocessing import StandardScaler # importando função para a Padronização dos dados
import seaborn as sns #pacote seaborn para construção de gráficos
import numpy as np #pacote numérico numpy

C:\Users\ohbto\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Importando arquivo de treino

In [2]:
treino = pd.read_csv("train.csv",header = 0)
#Removendo a coluna ID, pois ela não agrega informação
treino = treino.iloc[:,1:]
treino

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76015,2,48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60926.490000,0
76016,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,118634.520000,0
76017,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74028.150000,0
76018,2,25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84278.160000,0


In [3]:
#Vendo o nome das colunas
np.sort(treino.columns.tolist())

array(['TARGET', 'delta_imp_amort_var18_1y3', 'delta_imp_amort_var34_1y3',
       'delta_imp_aport_var13_1y3', 'delta_imp_aport_var17_1y3',
       'delta_imp_aport_var33_1y3', 'delta_imp_compra_var44_1y3',
       'delta_imp_reemb_var13_1y3', 'delta_imp_reemb_var17_1y3',
       'delta_imp_reemb_var33_1y3', 'delta_imp_trasp_var17_in_1y3',
       'delta_imp_trasp_var17_out_1y3', 'delta_imp_trasp_var33_in_1y3',
       'delta_imp_trasp_var33_out_1y3', 'delta_imp_venta_var44_1y3',
       'delta_num_aport_var13_1y3', 'delta_num_aport_var17_1y3',
       'delta_num_aport_var33_1y3', 'delta_num_compra_var44_1y3',
       'delta_num_reemb_var13_1y3', 'delta_num_reemb_var17_1y3',
       'delta_num_reemb_var33_1y3', 'delta_num_trasp_var17_in_1y3',
       'delta_num_trasp_var17_out_1y3', 'delta_num_trasp_var33_in_1y3',
       'delta_num_trasp_var33_out_1y3', 'delta_num_venta_var44_1y3',
       'imp_amort_var18_hace3', 'imp_amort_var18_ult1',
       'imp_amort_var34_hace3', 'imp_amort_var34_ult1',
   

## Redução de Dimensionalidade

In [4]:
## Ao todo, temos 370 variáveis a serem consideradas. Dessa forma, faremos uma redução de dimensionalidade
# baseado em uma randomForest para decidir pelas melhores variáveis
n_variaveis = 10

In [14]:
# Random Forest
RandomForest = ExtraTreesClassifier().fit(treino.iloc[:,0:369], treino.iloc[:,369])

C:\Users\ohbto\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [15]:
#Vamos selecionar as 10 variáveis de maior importância
melhores10_variaveis_RF = np.argsort(RandomForest.feature_importances_)[-n_variaveis:]
treino.columns[melhores10_variaveis_RF]

Index(['num_var22_ult3', 'num_var45_ult1', 'saldo_medio_var5_hace2',
       'num_var45_hace2', 'num_var45_hace3', 'num_var45_ult3',
       'saldo_medio_var5_ult3', 'saldo_medio_var5_hace3', 'var15', 'var38'],
      dtype='object')

In [17]:
treinoRF = treino.iloc[:,np.append(melhores10_variaveis_RF,len(treino.columns)-1)]
treinoRF

,num_var22_ult3,num_var45_ult1,saldo_medio_var5_hace2,num_var45_hace2,num_var45_hace3,num_var45_ult3,saldo_medio_var5_ult3,saldo_medio_var5_hace3,var15,var38,TARGET
0,0,0,0.00,0,0,0,0.00,0.00,23,39205.170000,0
1,0,0,0.00,0,0,0,0.00,88.89,34,49278.030000,0
2,0,0,3.00,0,0,0,2.07,0.18,23,67333.770000,0
3,3,18,186.09,27,3,48,138.84,0.00,37,64007.970000,0
4,9,0,3.00,0,0,0,13501.47,0.30,39,117310.979016,0
...,...,...,...,...,...,...,...,...,...,...,...
76015,0,0,0.00,0,0,0,0.00,0.00,48,60926.490000,0
76016,24,9,130.65,36,3,48,0.00,0.00,39,118634.520000,0
76017,0,0,3.00,0,0,0,3.00,0.00,23,74028.150000,0
76018,0,0,3.00,0,0,0,2.58,1.74,25,84278.160000,0


## Análise Exploratória dos dados

In [18]:
#Anterando os nomes das variáveis, apenas para facilitar os nomes
treinoRF.columns = np.append([''.join(('var',str(i))) for i in range(0,len(treinoRF.columns)-1)],'target')

In [19]:
treinoRF.describe()

,var0,var1,var2,var3,var4,var5,var6,var7,var8,var9,target
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000,7.602000e+04,76020.000000
mean,3.044238,4.363496,1579.135311,5.393212,3.894396,13.651105,1048.856447,8.913659e+02,33.212865,1.172358e+05,0.039569
std,6.206116,14.406485,12148.452398,14.496095,10.416821,33.304012,8189.948852,9.888597e+03,12.956486,1.826646e+05,0.194945
min,0.000000,0.000000,-128.370000,0.000000,0.000000,0.000000,-476.070000,-8.040000e+00,5.000000,5.163750e+03,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,23.000000,6.787061e+04,0.000000
50%,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,2.730000,9.900000e-01,28.000000,1.064092e+05,0.000000
75%,3.000000,3.000000,90.000000,3.000000,3.000000,12.000000,83.790000,1.221750e+01,40.000000,1.187563e+05,0.000000
max,234.000000,510.000000,812137.260000,342.000000,339.000000,801.000000,544365.570000,1.542339e+06,105.000000,2.203474e+07,1.000000


Vamos criar uma função que seja capaz de descrever as variáveis como describe e com mais alguns detalhes

In [20]:
# Criando uma função que retorna um dataframe de descrição de dados (tal qual a função describe do pacote explore do R)
def explore_describe(df):
    df_out = pd.DataFrame(columns = ['variable','type','na' ,'na_pct' ,'unique','min', 'quat25','median','mean', \
                                     'quat75','max','std','skewness','kurtosis','media_desvio'])
    df_out['variable'] = df.columns
    df_out['type'] = df.dtypes.values
    df_out['na'] = [len(df[coluna]) - df[coluna].count() for coluna in df.columns]
    df_out['na_pct'] = df_out.na/len(df_out.na)*100
    df_out['unique'] = [len(df[coluna].unique()) for coluna in df.columns]
    df_out['min']  = [min(df[coluna]) if 'int' in str(df[coluna].dtype) or 'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['mean'] = [round(df[coluna].mean(),2) if 'int' in str(df[coluna].dtype) or \
                      'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['max']  = [max(df[coluna]) if 'int' in str(df[coluna].dtype) or 'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['std'] = [round(df[coluna].std(),2) if 'int' in str(df[coluna].dtype) or \
                      'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['quat25'] = [round(df[coluna].quantile(0.25),2) if 'int' in str(df[coluna].dtype) or \
                      'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['quat75'] = [round(df[coluna].quantile(0.75),2) if 'int' in str(df[coluna].dtype) or \
                      'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['median'] = [round(df[coluna].quantile(0.5),2) if 'int' in str(df[coluna].dtype) or \
                      'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['skewness'] = [df[coluna].skew() if 'int' in str(df[coluna].dtype) or \
                          'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['kurtosis'] = [df[coluna].kurt() if 'int' in str(df[coluna].dtype) or \
                          'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['media_desvio'] = [round(df[coluna].mean()/df[coluna].std(),2) if 'int' in str(df[coluna].dtype) or \
                                'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    return(df_out)


Algumas conclusões podemos tirar dos dados:

  - *var0*: a média é maior que o 3º quartil. Podemos concluir que, por mais que a maioria dos dados estejam próximos a 0, existem *outliers* carregam a média para cima. <font color=blue>Verificar os outliers</font>
  - A distribuição de todas as variáveis possuem altos *outliers*, como é apresentado na figura abaixo, criado por seaborn
  - *var0* e *var5* possuem uma dependencia linear (correlação máxima). Portanto, não há necessidade da existencia de uma delas. <font color=blue>Remover var5</font>
  - Correlação positiva entre *var0* e *var3*
  - *var8* possui a maior taxa entre média e desvio padrão, mas possui um historgrama com poucos *outliers*
  - Pelos diagramas de dispersão, a forma de separação entre as variáveis é quase linear entre algumas duplas. <font color=blue>Testar SVM</font>
  - A maior dispersão encontra-se na var9

In [21]:
#Explorando as variaveis
explore_describe(treinoRF)

,variable,type,na,na_pct,unique,min,quat25,median,mean,quat75,max,std,skewness,kurtosis,media_desvio
0,var0,int64,0,0.0,33,0.00,0.00,0.00,3.04,3.00,234.00,6.21,4.209854,50.330588,0.49
1,var1,int64,0,0.0,94,0.00,0.00,0.00,4.36,3.00,510.00,14.41,8.643107,132.220842,0.30
2,var2,float64,0,0.0,14486,-128.37,0.00,3.00,1579.14,90.00,812137.26,12148.45,22.400752,863.277555,0.13
3,var3,int64,0,0.0,85,0.00,0.00,0.00,5.39,3.00,342.00,14.50,6.454405,66.617128,0.37
4,var4,int64,0,0.0,66,0.00,0.00,0.00,3.89,3.00,339.00,10.42,6.876337,93.456969,0.37
5,var5,int64,0,0.0,172,0.00,0.00,0.00,13.65,12.00,801.00,33.30,6.404366,69.025601,0.41
6,var6,float64,0,0.0,17330,-476.07,0.00,2.73,1048.86,83.79,544365.57,8189.95,26.620541,1153.343844,0.13
7,var7,float64,0,0.0,7787,-8.04,0.00,0.99,891.37,12.22,1542339.36,9888.60,70.032179,9125.546556,0.09
8,var8,int64,0,0.0,100,5.00,23.00,28.00,33.21,40.00,105.00,12.96,1.578367,2.519919,2.56
9,var9,float64,0,0.0,57736,5163.75,67870.61,106409.16,117235.81,118756.25,22034738.76,182664.60,51.274532,4219.873445,0.64


In [ ]:
sns.pairplot(treinoRF, hue = "target")

C:\Users\ohbto\Anaconda3\lib\site-packages\statsmodels\nonparametric\kde.py:487: RuntimeWarning: invalid value encountered in true_divide
  binned = fast_linbin(X, a, b, gridsize) / (delta * nobs)
C:\Users\ohbto\Anaconda3\lib\site-packages\statsmodels\nonparametric\kdetools.py:34: RuntimeWarning: invalid value encountered in double_scalars
  FAC1 = 2*(np.pi*bw/RANGE)**2


Vamos verificar a matriz de correlação entre as variáveis
 - Pela matriz de correlação, nota-se que a correlação entre as variáveis é máxima (1). Dessa forma, podemos remover uma das variáveis

In [ ]:
sns.heatmap(treinoRF.corr(), annot=True, fmt=".2f",cmap="YlGnBu")

Vamos confirmar, a partir de bloxplots, a existencia de *outliers*

In [ ]:
sns.boxenplot(data = treinoRF[['var0','var2','var3','var4','var8']])

In [ ]:
sns.boxenplot(data = treinoRF.drop(columns = ['var0','var2','var3','var4','var8','var9','target']))

In [ ]:
sns.boxenplot(data = treinoRF[['var9']])

In [ ]:
#Removendo var05
treinoRF = treinoRF.drop(columns=['var5'])

## Tratamento dos dados: remoção dos *outliers*

Nota-se um volume muito alto de outliers na base de dados. Para facilitar a modelagem, vamos tratá-los

In [ ]:
#Vamos verificar quantos sao os dados, de cada variavel, que estão acima de 3 desvios padrões
desvio = explore_describe(treinoRF).set_index('variable')['std']
quat75 = explore_describe(treinoRF).set_index('variable')['quat75']
quat75

# <font color = red> Verificar esses 26%</font>

In [ ]:
#Este resultado mostra a quantidade de outliers dentro do dataset treinoRF (percentualmente)
pd.DataFrame({'Porcent3desvios':[100*(len(treinoRF[treinoRF[x]>=3*desvio[x]]))/len(treinoRF[x]) \
                                 for x in quat75.index]}).set_index(desvio.index).iloc[0:-1,:]

In [ ]:
treinoRF_Padronizados = StandardScaler().fit(treinoRF.drop(columns = ['target'])).transform(treinoRF.drop(columns = ['target']))
treinoRF_Padronizados = pd.DataFrame(treinoRF_Padronizados, columns = [' '.join(('var_pad',str(i))) for i in range(0,len(treinoRF.columns)-1)])

In [ ]:
treinoRF_Padronizados.hist()